In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from torchmetrics.functional import r2_score as r2_loss
from sklearn.model_selection import train_test_split
from itertools import product

import torch
import torch.nn as nn
import torch.optim as optim
import sys 
import os
from torch.utils.data import DataLoader, TensorDataset

sys.path.append(os.path.dirname(os.getcwd())) 
from source.autoencoder import VariationalAutoencoder

### Simulate time series with intervals 
Now time series array of train data consists of 3 parts: mean values of daily incidence, low valuee of daily incidence and high values of daily incidence for each day. Therefore, the size of time series array is three times bigger than tmax -- number of simulation days. 

In [8]:
df = pd.read_csv('../data/network_model_interval_dataset_72.csv', index_col=0)
tmax = int((df.values.shape[1] - 5)/3) # number of simulation days
# 5 - number of parameters of the network model
df['ts'] = df[['incidence_' + str(day_index) for day_index in range(tmax)] + \
              ['low_incidence_' + str(day_index) for day_index in range(tmax)] + \
              ['high_incidence_' + str(day_index) for day_index in range(tmax)]].values.tolist()
data = df[['beta', 'alpha', 'ts']]

In [9]:
print(r'Диапазон параметров beta:[{}, {}], alpha:[{}, {}]'.format(df['beta'].min(), df['beta'].max(),
                                                                             df['alpha'].min(),
                                                                             df['alpha'].max()))

Диапазон параметров beta:[0.1, 0.9], alpha:[0.2, 0.9000000000000002]


In [10]:
X_train, X_test, y_train, y_test =  train_test_split(data.drop(columns=['ts']), 
                                                     data.ts.values, 
                                                     test_size=0.33, 
                                                     random_state=42)
X_train, X_test = X_train.values, X_test.values